# Informe Final: Implementación de QAOA Básico para Max-Cut

**Autores:** Mauro Castillo, Javiera Osorio, Mariano Varas Ramos

**Ramo:** INF456 - Introducción a la Computación Cuántica

**Fecha:** 2 de Diciembre, 2025

## 1. Descripción del Problema que Estamos Implementando

Estamos implementando el **Quantum Approximate Optimization Algorithm (QAOA)** para resolver el problema de **Max-Cut**, el cual es un *benchmark* estándar NP-hard en optimización combinatoria. El objetivo es validar la funcionalidad de QAOA en hardware NISQ y explorar técnicas avanzadas para escalar su rendimiento y mitigar las dificultades inherentes al hardware.

Nuestra implementación se basa en los tutoriales de Qiskit [1, 2], siguiendo un flujo de trabajo de 4 pasos:

### 1) Flujo de Trabajo de QAOA Base

**Paso 1: Mapeo Clásico $\rightarrow$ Cuántico**
1.  **Grafo $\rightarrow$ Hamiltoniano:** El problema (un grafo $G=(V, E)$) se convierte en un Hamiltoniano de costo $H_C$. Para Max-Cut, cada arista $(i, j)$ se traduce en un término de Pauli $Z_i Z_j$. Buscamos el estado (bitstring) que minimiza $\langle H_C \rangle$.
    * *Implementación:* Usamos `SparsePauliOp.from_sparse_list` [2, celda 3].

**Paso 2: Construcción y Optimización del Circuito**
1.  **Hamiltoniano $\rightarrow$ Ansatz:** Construimos el circuito variacional `QAOAAnsatz(reps=p)` [2, celda 4], que alterna $p$ capas del operador de costo $e^{-i\gamma_k H_C}$ y el mezclador $e^{-i\beta_k H_B}$.
2.  **Transpilación:** El circuito lógico se transpila usando `generate_preset_pass_manager` (level 3) para adaptarlo a la conectividad y compuertas nativas del backend [2, celda 6].

**Paso 3: Ejecución y Optimización (Bucle Híbrido)**
1.  **Primitiva `Estimator`:** Se usa el `Estimator` de Qiskit Runtime para calcular el valor esperado $\langle H_C \rangle$ [2, celda 8].
2.  **Optimizador Clásico:** Un optimizador (ej. COBYLA) ajusta los parámetros $(\vec{\gamma}, \vec{\beta})$ iterativamente para encontrar el mínimo de $\langle H_C \rangle$ [2, celda 9].

**Paso 4: Post-procesamiento y Solución**
1.  **Primitiva `Sampler`:** Con los parámetros óptimos $(\vec{\gamma}^*, \vec{\beta}^*)$, se usa el `Sampler` para obtener la distribución de probabilidad de las soluciones [2, celda 12].
2.  **Resultado:** Se extrae el bitstring más probable o el de menor energía como la solución candidata al problema de Max-Cut [2, celda 13].

### 2) Flujo de Trabajo Completo del Pipeline QAOA + CVaR (Implementación N=20)

#### 1) Construcción del Problema Max-Cut
**Paso 1: Generación del Grafo**  
Se crea un grafo aleatorio manual para $N=20$ nodos con probabilidad $p=0.15$.  
* *Implementación:* `rustworkx.PyGraph()`.

**Paso 2: Hamiltoniano de Costo**  
Cada arista $(i, j)$ se convierte en un término Pauli $Z_i Z_j$. El Hamiltoniano final es un `SparsePauliOp`.  
* *Implementación:* `build_max_cut_paulis`.

#### 2) Construcción del Ansatz QAOA
**Paso 1: Definición del Circuito**  
Se instancia `QAOAAnsatz` con el operador de costo y una sola capa ($p=1$).

**Paso 2: Mediciones Explícitas**  
Se añaden mediciones (`measure_all()`) al final del ansatz para que la primitiva *Sampler* pueda registrar los bitstrings resultantes.

#### 3) Mapeo de Qubits vía SAT (SATMapper)
**Paso 1: Modelo SAT**  
Se modela el problema de asignación física considerando la topología lineal de 25 qubits. Se restringe que dos qubits lógicos no compartan uno físico y que las compuertas lógicas correspondan a aristas físicas válidas.  
* *Implementación:* Clase `SATMapper` con `pysat`.

**Paso 2: Solución del Layout**  
El solver SAT devuelve un mapeo (`initial_layout`) garantizando conectividad perfecta para el circuito.

#### 4) Transpilación del Circuito
**Paso 1: Adaptación al Hardware**  
Se transpila el circuito aplicando el `initial_layout` obtenido por SAT y optimizando las compuertas para el backend simulado.  
* *Implementación:* `transpile(..., optimization_level=3)`.

#### 5) Ejecución y Optimización (Bucle Híbrido)
**Paso 1: Muestreo (Sampler)**  
Se usa `AerSampler` (SamplerV2) para ejecutar el circuito transpilado con los parámetros actuales y obtener cuentas de bitstrings (2048 shots).  
* *Implementación:* `local_sampler.run()`.

**Paso 2: Función de Costo CVaR**  
En lugar del valor esperado estándar, se calcula el **CVaR** ($\alpha=0.25$). Se promedian solo las mejores soluciones (la cola de la distribución) para acelerar la convergencia hacia el óptimo.  
* *Implementación:* `cvar_cost_from_probabilities`.

**Paso 3: Optimización Clásica**  
El algoritmo **COBYLA** minimiza la función de costo CVaR iterativamente (max 15 iteraciones) ajustando los parámetros $\vec{\gamma}, \vec{\beta}$.  
* *Implementación:* `scipy.optimize.minimize`.

#### 6) Resultados
**Paso 1: Evaluación Final**  
Se extrae el mejor costo encontrado y se visualiza la curva de convergencia del optimizador.

---

## 2. Avance a la Fecha: Implementación Base (N=5 Nodos)

Hemos implementado y ejecutado con éxito el **flujo completo de QAOA Base** (descrito en la sección anterior) para un grafo pequeño de 5 nodos, siguiendo el tutorial [2]. Esto nos ha permitido validar cada paso del proceso.

### 2.1. Instalación de Dependencias
Primero, instalamos las librerías necesarias.

In [ ]:
!pip install 'qiskit[visualization]' -q
!pip install qiskit-ibm-runtime -q
!pip install rustworkx -q
!pip install python-sat -q
!pip install qiskit-aer -q

In [ ]:
!pip uninstall numpy -y -q
!pip install numpy==1.26.4 scipy==1.12.0 -q
!pip install qiskit==1.1.1 qiskit-aer==0.14.2 -q
!pip install rustworkx==0.14.2 python-sat==0.1.8.dev1 matplotlib -q

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import rustworkx as rx
from rustworkx.visualization import mpl_draw as draw_graph
import numpy as np
from scipy.optimize import minimize
from collections import defaultdict
from typing import Sequence, Callable, Iterable
from dataclasses import dataclass
from itertools import combinations
from threading import Timer

from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz
from qiskit.circuit import QuantumCircuit, ParameterVector
from qiskit.transpiler import CouplingMap, PassManager
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.passes.routing.commuting_2q_gate_routing import (
    SwapStrategy,
    FindCommutingPauliEvolutions,
    Commuting2qGateRouter,
)

from pysat.formula import CNF, IDPool
from pysat.solvers import Solver

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler

print("Librerías importadas correctamente.")

### 2.2. Paso 1: Mapeo Grafo $\rightarrow$ Hamiltoniano (N=5)

In [ ]:
n = 5
graph_5 = rx.PyGraph()
graph_5.add_nodes_from(np.arange(0, n, 1))
edge_list_5 = [
    (0, 1, 1.0),
    (0, 2, 1.0),
    (0, 4, 1.0),
    (1, 2, 1.0),
    (2, 3, 1.0),
    (3, 4, 1.0),
]
graph_5.add_edges_from(edge_list_5)

print("Grafo de N=5 nodos:")
draw_graph(graph_5, node_size=600, with_labels=True)

In [ ]:
def build_max_cut_paulis(graph: rx.PyGraph) -> list[tuple[str, float]]:
    pauli_list = []
    for edge in list(graph.edge_list()):
        weight = graph.get_edge_data(edge[0], edge[1])
        pauli_list.append(("ZZ", [edge[0], edge[1]], weight))
    return pauli_list

max_cut_paulis_5 = build_max_cut_paulis(graph_5)
cost_hamiltonian_5 = SparsePauliOp.from_sparse_list(max_cut_paulis_5, n)

print("Cost Function Hamiltonian (H_C) para N=5:")
print(cost_hamiltonian_5.paulis.to_labels())

### 2.3. Pasos 2-4: Ansatz, Optimización y Resultados (N=5)

Construimos el `QAOAAnsatz` con $p=2$ capas, definimos la función de costo y ejecutamos el bucle de optimización.

A diferencia del tutorial [2], usaremos `qiskit_aer` (un simulador local de alto rendimiento) para ejecutar este ejemplo base. Esto es mucho más rápido para la depuración y no requiere una conexión a la nube de IBM.

In [ ]:
# --- 1. backend de aersimulator ---
from qiskit_aer.primitives import Estimator as AerEstimator, Sampler as AerSampler
from qiskit_aer import AerSimulator

print("Usando simulador local (AerSimulator) para esta implementación.")
local_backend = AerSimulator()
pm = generate_preset_pass_manager(optimization_level=1, backend=local_backend) # nivel 1 es suficiente para simulador

# --- 2. construcción y transpilación del tircuito ---
circuit_5 = QAOAAnsatz(cost_operator=cost_hamiltonian_5, reps=2)
circuit_5.measure_all()
candidate_circuit_5 = pm.run(circuit_5)

print(f"Circuito N=5, p=2 transpiled. Profundidad: {candidate_circuit_5.depth()}")

# --- 3. definición de costo y optimización ---
objective_func_vals = []
def cost_func_estimator(params, ansatz, hamiltonian, estimator):
    job = estimator.run(
        circuits=[ansatz],
        observables=[hamiltonian],
        parameter_values=[params]
    )
    results = job.result()
    cost = results.values[0]
    objective_func_vals.append(cost)
    return cost

init_params = [np.pi, np.pi / 2] * 2 # Para p=2

estimator = AerEstimator()
estimator.options.default_shots = 1024

result = minimize(
    cost_func_estimator,
    init_params,
    args=(candidate_circuit_5, cost_hamiltonian_5, estimator),
    method="COBYLA",
    tol=1e-2,
)
print("Optimización finalizada:")
print(result)

In [ ]:
# --- 4. Post-procesamiento y Muestreo ---

optimized_circuit_5 = candidate_circuit_5.assign_parameters(result.x)

sampler = AerSampler()

shots_num = 10000

job = sampler.run(circuits=[optimized_circuit_5], shots=shots_num)
results = job.result()

quasi_dist = results.quasi_dists[0]
final_distribution_bin = quasi_dist.binary_probabilities()

print(f"Distribución de probabilidades (primeros 5): {dict(list(final_distribution_bin.items())[:5])}")

counts_bin = {k: v * shots_num for k, v in final_distribution_bin.items()}
shots = sum(counts_bin.values())

### 2.4. Resultados Obtenidos (N=5)

El flujo se ejecutó correctamente. El optimizador convergió (ver Fig. 1 Izq.), y la distribución final (Fig. 1 Centro) identifica las soluciones óptimas (`10100` y `01011` o sus opuestos bit a bit) para este grafo. (Nota: los bitstrings de Qiskit se leen de derecha a izquierda, `00101` $\equiv$ `10100`).

In [ ]:
# --- visualización de resultados ---

def to_bitstring_from_str(str_bits):
    return [int(digit) for digit in str_bits]

def evaluate_sample(x: Sequence[int], graph: rx.PyGraph) -> float:
    # función de costo de Max-Cut (maximizar cortes)
    cut_size = 0
    for u, v in list(graph.edge_list()):
        if x[u] != x[v]:
            cut_size += 1
    return cut_size

def plot_result_graph(G, x):
    colors = ["#FFCA00" if i == 0 else "#0064a2" for i in x] # colores usm xd
    pos = rx.spring_layout(G)
    plt.figure(figsize=(6, 4))
    rx.visualization.mpl_draw(
        G, node_color=colors, node_size=600, alpha=0.9, pos=pos, with_labels=True, node_shape='o'
    )
    plt.show()

# graficar convergencia
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(objective_func_vals)
4.2. Escalación a N=100 (y má
plt.xlabel("Iteración")
plt.ylabel("Costo (Energía)")
plt.title("Convergencia del Optimizador (N=5)")

# graficar distribución final
plt.subplot(1, 2, 2)
keys_sorted = sorted(final_distribution_bin, key=final_distribution_bin.get, reverse=True)
values_sorted = [final_distribution_bin[k] for k in keys_sorted]
plt.xticks(rotation=90)
plt.title(f"Distribución de Resultados (N=5, {shots} shots)")
plt.xlabel("Bitstrings")
plt.ylabel("Probabilidad")
plt.bar(keys_sorted, values_sorted, color="tab:grey")
# resaltar las 2 soluciones óptimas (y sus opuestos)
optimal_strs = ['01011', '10100']
for i, key in enumerate(keys_sorted):
    if key in optimal_strs:
        plt.gca().get_children()[i].set_color("tab:purple")
plt.tight_layout()
plt.show()

# --- graficar mejor corte ---
most_likely_str = keys_sorted[0]
most_likely_bitstring = to_bitstring_from_str(most_likely_str)

print(f"Bitstring más probable: {most_likely_str}")
print(f"Valor del corte: {evaluate_sample(most_likely_bitstring, graph_5)}")
plot_result_graph(graph_5, most_likely_bitstring)

---

#Avances de implementación

### 3.0 Configuración del Entorno
Importamos las librerías necesarias: `Qiskit` para el circuito, `Rustworkx` para grafos, `SciPy` para optimización clásica y `PySAT` para resolver el mapeo de qubits.

In [ ]:

import numpy as np
import rustworkx as rx
import matplotlib.pyplot as plt
from time import perf_counter
from scipy.optimize import minimize

from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz
from qiskit.transpiler import PassManager, CouplingMap
from qiskit.compiler import transpile
from qiskit_aer.primitives import SamplerV2 as AerSampler

from pysat.formula import CNF, IDPool
from pysat.solvers import Solver

print("Librerías importadas correctamente.")

### 3.0.1 Definición de Funciones Auxiliares y Clases
Aquí definimos las herramientas fundamentales para el flujo de trabajo:
1.  **`build_max_cut_paulis`**: Convierte el grafo en un Operador de Pauli (Hamiltoniano).
2.  **`cvar_cost_from_probabilities`**: Calcula el valor de riesgo condicional (CVaR) para optimizar la cola de la distribución.
3.  **`SATMapper`**: Una clase que utiliza lógica booleana (SAT) para encontrar un mapeo físico perfecto de los qubits en una topología lineal.

In [ ]:
# --- Funciones de Costo y Utilidad ---
def build_max_cut_paulis(graph: rx.PyGraph) -> SparsePauliOp:
    pauli_list = []
    num_nodes = len(graph.node_indices())
    for edge in graph.edge_list():
        pauli_str = ['I'] * num_nodes
        pauli_str[edge[0]] = 'Z'
        pauli_str[edge[1]] = 'Z'
        pauli_list.append("".join(pauli_str))
    return SparsePauliOp(pauli_list)

def evaluate_max_cut(bitstring: str, graph: rx.PyGraph) -> float:
    cut_size = 0
    bits_rev = bitstring[::-1] # Qiskit usa little-endian
    for u, v in graph.edge_list():
        if bits_rev[u] != bits_rev[v]:
            cut_size += 1
    return cut_size

def cvar_cost_from_probabilities(probabilities, graph, alpha=0.1):
    # Ordenar bitstrings por valor de corte (de mayor a menor)
    sorted_probs = sorted(probabilities.items(), key=lambda item: evaluate_max_cut(item[0], graph), reverse=True)
    cvar_value = 0.0
    accumulated_prob = 0.0
    for bitstring, prob in sorted_probs:
        if accumulated_prob < alpha:
            prob_to_add = min(prob, alpha - accumulated_prob)
            cut_value = evaluate_max_cut(bitstring, graph)
            cvar_value += prob_to_add * cut_value
            accumulated_prob += prob_to_add
        else:
            break
    return cvar_value / accumulated_prob if accumulated_prob > 0 else 0

# --- Clase SATMapper ---
class SATMapper:
    def __init__(self, coupling_map):
        self.cmap = coupling_map
        self.n_phys = len(coupling_map.physical_qubits)
        self.id_pool = IDPool()
        self._var = lambda l, p: self.id_pool.id(f"x_{l}_{p}")

    def find_mapping(self, logical_edges, num_logical_qubits):
        cnf = CNF()
        # 1. Cada qubit lógico a exactamente un físico (simplificado a al menos uno aquí, unicidad implícita)
        for l in range(num_logical_qubits):
            cnf.append([self._var(l, p) for p in range(self.n_phys)])
            # Unicidad (opcional para reducir búsqueda pero estricta para corrección)
            for p1 in range(self.n_phys):
                for p2 in range(p1 + 1, self.n_phys):
                    cnf.append([-self._var(l, p1), -self._var(l, p2)])

        # 2. Dos qubits lógicos no pueden estar en el mismo físico
        for p in range(self.n_phys):
            for l1 in range(num_logical_qubits):
                for l2 in range(l1 + 1, num_logical_qubits):
                    cnf.append([-self._var(l1, p), -self._var(l2, p)])

        # 3. Restricción de conectividad para las aristas lógicas
        for i, j in logical_edges:
            clauses = []
            for p, q in self.cmap.get_edges():
                aux_forward = self.id_pool.id(f"e_{i}{p}_{j}{q}")
                aux_backward = self.id_pool.id(f"e_{i}{q}_{j}{p}")
                # Si (i->p) y (j->q), entonces activamos aux
                cnf.extend([[-aux_forward, self._var(i,p)], [-aux_forward, self._var(j,q)]])
                cnf.extend([[-aux_backward, self._var(i,q)], [-aux_backward, self._var(j,p)]])
                clauses.extend([aux_forward, aux_backward])
            cnf.append(clauses) # Al menos una arista física válida debe ser usada

        with Solver(bootstrap_with=cnf) as solver:
            if not solver.solve(): return None
            model = solver.get_model()

        mapping = {}
        for v in model:
            if v > 0:
                name = self.id_pool.obj(v)
                if name and name.startswith('x_'):
                    _, l, p = name.split('_')
                    mapping[int(l)] = int(p)
        return mapping

print("Funciones y Clase SATMapper definidas.")

### 3.1. Construcción del Problema Max-Cut
Generamos un grafo aleatorio Erdős–Rényi con **$N=20$ nodos** y probabilidad de arista $p=0.15$. Posteriormente, transformamos este grafo en un Hamiltoniano de costo ($H_C$) compuesto por términos $Z_i Z_j$.

In [ ]:
print("\n--- Paso 1: Configurando problema para N=20 ---")
n_adv = 20
np.random.seed(123)
prob = 0.15

# Grafo aleatorio
graph_adv = rx.PyGraph()
graph_adv.add_nodes_from(range(n_adv))
edge_list = []
for i in range(n_adv):
    for j in range(i + 1, n_adv):
        if np.random.random() < prob:
            edge_list.append((i,j))
graph_adv.add_edges_from_no_data(edge_list)

# Definición del Hardware (Simulado)
num_physical_qubits = 25
line_cmap = CouplingMap.from_line(num_physical_qubits)

# Hamiltoniano de Costo
cost_hamiltonian_adv = build_max_cut_paulis(graph_adv)
print(f"Grafo N={n_adv} generado con {len(edge_list)} aristas.")

###  3.2. Construcción del Ansatz QAOA
Creamos el circuito variacional QAOA con una profundidad de $p=1$.
**Nota Clave:** Añadimos `measure_all()` explícitamente para que la primitiva `Sampler` pueda registrar los bitstrings de salida necesarios para calcular el costo CVaR.

In [ ]:
# QAOA con p=1
ansatz_adv = QAOAAnsatz(cost_operator=cost_hamiltonian_adv, reps=1)

# IMPORTANTE: Mediciones para el Sampler
ansatz_adv.measure_all()

print("Ansatz QAOA construido y mediciones añadidas.")

###  3.3. Mapeo de Qubits vía SAT (SATMapper)
Dado que el hardware simulado tiene una topología lineal restringida, utilizamos un solver SAT para encontrar un `initial_layout` válido. Esto garantiza que todas las compuertas de dos qubits del circuito lógico correspondan a conexiones físicas existentes.

In [ ]:
print("--- Paso 3: Buscando mapeo SAT ---")
sat_mapper = SATMapper(line_cmap)

# Extraer aristas lógicas del circuito (qubits que interactúan)
logical_edges = [(q, p) for op, qubits, _ in ansatz_adv.decompose().data if len(qubits)==2 for q,p in [qubits]]

# Ejecutar Solver
initial_layout_int = sat_mapper.find_mapping(logical_edges, n_adv)

if not initial_layout_int:
    raise ValueError("No se encontró mapeo SAT. Deteniendo ejecución.")

# Convertir formato int -> Qubit object
logical_qubits = ansatz_adv.qubits
initial_layout = {logical_qubits[k]: v for k, v in initial_layout_int.items()}
print("Mapeo SAT encontrado con éxito.")

###  3.4. Transpilación del Circuito
Compilamos el circuito lógico adaptándolo al hardware físico (simulado). Utilizamos el layout encontrado por el `SATMapper` y un nivel de optimización alto (3) para reducir la profundidad y el ruido.

In [ ]:
print("--- Paso 4: Transpilando circuito ---")
start_time = perf_counter()

transpiled_circuit = transpile(
    ansatz_adv,
    initial_layout=initial_layout,
    coupling_map=line_cmap,
    optimization_level=3
)

t_time = perf_counter() - start_time
print(f"Transpilación completada en {t_time:.2f}s.")
print(f"Profundidad: {transpiled_circuit.depth()}, CNOTs: {transpiled_circuit.count_ops().get('cx', 0)}")

###  3.5. Ejecución y Optimización (Bucle Híbrido)
Ejecutamos el bucle clásico-cuántico:
1.  **Cuántica:** El `Sampler` ejecuta el circuito con parámetros $(\gamma, \beta)$.
2.  **Clásica:** Calculamos el costo **CVaR** ($\alpha=0.25$) basándonos en las probabilidades de los bitstrings.
3.  **Optimizador:** `COBYLA` ajusta los parámetros para minimizar este costo.

In [ ]:
print("\n--- Paso 5: Iniciando Optimización (QAOA + CVaR) ---")

objective_vals_cvar = []
local_sampler = AerSampler()

def cost_func_cvar(params):
    # Asignar parámetros al circuito transpilado
    bound_circuit = transpiled_circuit.assign_parameters(params)

    # Ejecutar en simulador
    job = local_sampler.run([(bound_circuit,)], shots=2048)

    # Obtener resultados
    # Nota: SamplerV2 devuelve PubResult, accedemos a data.meas
    counts = job.result()[0].data.meas.get_counts()
    total_shots = sum(counts.values())

    # Calcular probabilidades y CVaR
    probs = {bs: c / total_shots for bs, c in counts.items()}
    cost = cvar_cost_from_probabilities(probs, graph_adv, alpha=0.25)

    objective_vals_cvar.append(cost)
    print(f"   Iter {len(objective_vals_cvar)}: Costo CVaR = {cost:.4f}")
    return -cost # Minimizamos el negativo del corte (maximizamos corte)

# Parámetros iniciales aleatorios
init_params_adv = np.random.rand(ansatz_adv.num_parameters) * np.pi

# Ejecutar optimizador
res_cvar = minimize(
    cost_func_cvar,
    init_params_adv,
    method='COBYLA',
    options={'maxiter': 15}
)
print("Optimización finalizada.")

###  3.6. Resultados y Visualización
Mostramos el mejor costo encontrado por el algoritmo y graficamos la curva de convergencia para visualizar cómo QAOA mejoró la solución iteración tras iteración.

In [ ]:
print(f"Grafo aleatorio (manual) N={n_adv} y topología de línea de {num_physical_qubits} qubits listos.")

print("\nTranspilando circuito N=20 con SATMapper... (puede tardar un momento)")
start_time_transpile = perf_counter()
sat_mapper = SATMapper(line_cmap)
logical_edges = [(q, p) for op, qubits, _ in ansatz_adv.decompose().data if len(qubits)==2 for q,p in [qubits]]
initial_layout_int = sat_mapper.find_mapping(logical_edges, n_adv)

if not initial_layout_int:
    print("ADVERTENCIA: No se encontró mapeo SAT. No se puede continuar.")
else:
    logical_qubits = ansatz_adv.qubits
    initial_layout = {logical_qubits[k]: v for k, v in initial_layout_int.items()}

    transpiled_circuit = transpile(ansatz_adv, initial_layout=initial_layout, coupling_map=line_cmap, optimization_level=3)
    transpile_time = perf_counter() - start_time_transpile

    print(f"Circuito N=20 transpilado en {transpile_time:.2f} segundos.")
    print(f"Profundidad: {transpiled_circuit.depth()}, CNOTs: {transpiled_circuit.count_ops().get('cx', 0)}")

    objective_vals_cvar = []
    local_sampler = AerSampler()

    def cost_func_cvar(params):
        bound_circuit = transpiled_circuit.assign_parameters(params)
        job = local_sampler.run([(bound_circuit,)], shots=2048)
        # El resultado ahora tendrá el registro 'meas' porque lo añadimos al circuito.
        counts = job.result()[0].data.meas.get_counts()
        total_shots = sum(counts.values())
        probs = {bs: c / total_shots for bs, c in counts.items()}
        cost = cvar_cost_from_probabilities(probs, graph_adv, alpha=0.25)
        objective_vals_cvar.append(cost)
        print(f"Iteración: {len(objective_vals_cvar)}, Costo CVaR: {cost:.4f}")
        return -cost

    init_params_adv = np.random.rand(ansatz_adv.num_parameters) * np.pi
    print("\nIniciando optimización para N=20 (CVaR)...")

    res_cvar = minimize(
        cost_func_cvar,
        init_params_adv,
        method='COBYLA',
        options={'maxiter': 15}
    )

    print("\n--- Optimización N=20 finalizada ---")
    print(res_cvar)

    plt.figure(figsize=(8, 5))
    plt.plot(objective_vals_cvar, marker='o', color='purple')
    plt.xlabel("Iteración del Optimizador")
    plt.ylabel("Costo CVaR (Tamaño del Corte)")
    plt.title("Convergencia de QAOA con CVaR (N=20 en Simulador)")
    plt.grid(True)
    plt.show()

    print(f"\nMejor valor de corte (CVaR) encontrado: {-res_cvar.fun:.2f}")

## 4. Dificultades Enfrentadas

El desarrollo de este proyecto, especialmente el escalado de la implementación base a las técnicas avanzadas, presentó una serie de desafíos técnicos significativos, principalmente relacionados con la configuración del entorno de software y la rápida evolución del ecosistema Qiskit.

1.  **Conflictos de Versiones y Compatibilidad del Entorno (Google Colab):**
    El principal obstáculo fue la gestión de dependencias. El entorno de Google Colab se actualiza constantemente, preinstalando versiones de librerías fundamentales como `numpy` y `scipy` (versiones >2.0 y >1.12 respectivamente). Estas versiones resultaron ser incompatibles con el conjunto de paquetes de `Qiskit 1.1.1` y sus dependencias (como `rustworkx 0.14.2`), que fueron construidas sobre versiones anteriores de `numpy`. Esto provocó:
    *   **Errores de Incompatibilidad Binaria:** Se manifestó el error `ValueError: numpy.dtype size changed`, que ocurre cuando una librería compilada con una versión de `numpy` se carga en un entorno que usa una versión diferente.
    *   **Advertencias Constantes del Resolvedor de `pip`:** Mensajes de `ERROR: pip's dependency resolver...` indicaban que nuestras instalaciones "rompían" la compatibilidad de otros paquetes preinstalados en Colab.
    *   **Solución:** La estrategia definitiva fue forzar la desinstalación de las versiones de `numpy` y `scipy` preinstaladas en Colab y luego instalar explícitamente las versiones exactas y compatibles (`numpy==1.26.4`, `scipy==1.12.0`) necesarias para nuestro stack de Qiskit. Este proceso requirió un reinicio manual del entorno de ejecución para asegurar que las nuevas librerías se cargaran correctamente en memoria.

2.  **Evolución y Fragmentación de la API de Qiskit:**
    A medida que el proyecto avanzaba, nos encontramos con que la documentación y los tutoriales a menudo se referían a diferentes versiones de Qiskit, donde la ubicación y el uso de módulos clave habían cambiado.
    *   **Importación de `Fake Providers`:** El módulo para simular backends de hardware (`FakeBrisbane`, `FakeManila`) fue movido. Inicialmente estaba en `qiskit.providers.fake_provider`, luego pasó a `qiskit_ibm_provider` y finalmente se integró en `qiskit_aer`. Esto causó errores de `ModuleNotFoundError`. La solución fue simplificar, eliminando la dependencia y creando una `CouplingMap` manualmente para simular una topología.
    *   **API de Primitives (V1 vs. V2):** El código inicial usaba la API de Primitives V1 (`Estimator`, `Sampler`). Sin embargo, las versiones más nuevas de `qiskit-aer` esperan la API V2 (`EstimatorV2`, `SamplerV2`). Esto provocó errores como `ImportError: cannot import name 'BaseEstimator'` y `TypeError` en los constructores de los Primitives. Fue necesario refactorizar todo el código que interactuaba con los simuladores para usar la nueva sintaxis de Primitives V2 (basada en "pubs").
    *   **Falta de Medidas en el Circuito:** Un error funcional clave fue olvidar añadir instrucciones de medida (`.measure_all()`) al `QAOAAnsatz`. Esto causó que el `SamplerV2` devolviera un resultado vacío, llevando a un `AttributeError: 'DataBin' object has no attribute 'meas'` al intentar acceder a los resultados de la medición.

3.  **Desafíos de Simulación a Gran Escala (N=100):**
    Al intentar escalar a 100 qubits, nos enfrentamos a las limitaciones inherentes de la simulación clásica:
    *   **Memoria Insuficiente:** El `AerSimulator` intentó usar por defecto el método `statevector`, que es imposible para 100 qubits. Esto generó un error de `Insufficient memory`.
    *   **Solución:** Fue necesario forzar explícitamente el uso de un método de simulación más eficiente en memoria, como `"matrix_product_state"`, pasándolo como opción al `Sampler` para permitir que la simulación se ejecutara.

En conclusión, la mayoría de las dificultades no estuvieron en la lógica del algoritmo cuántico en sí, sino en la ingeniería de software necesaria para construir un entorno de ejecución estable y adaptarse a la rápida evolución de las APIs de Qiskit.





---



## 5. Conclusiones

Este proyecto ha demostrado con éxito la viabilidad de escalar el algoritmo QAOA para el problema de Max-Cut, superando una implementación base (N=5) para abordar problemas de mayor envergadura (N=20) mediante la aplicación de técnicas avanzadas de transpilación y optimización. El objetivo principal, que era validar un pipeline completo y robusto ejecutable en simuladores locales, se ha cumplido satisfactoriamente.

**Principales Logros y Hallazgos:**

1.  **Validación de Técnicas Avanzadas:** Se implementaron y validaron con éxito dos técnicas clave para la escalabilidad de QAOA.
    *   **Transpilación Optimizada con `SATMapper`:** Se demostró que es posible encontrar un layout inicial de qubits que satisface las restricciones de conectividad de un hardware (simulado en este caso). Esto es fundamental para reducir la profundidad del circuito y minimizar la cantidad de compuertas SWAP, que son una fuente principal de errores en dispositivos NISQ. Para nuestro problema de N=20, logramos un circuito transpilado de profundidad reducida, validando la eficacia del método.
    *   **Optimización Robusta con CVaR:** La adopción de la función de costo CVaR (Conditional Value at Risk) en lugar del valor esperado estándar representó una mejora clave. Como se observa en la curva de convergencia para N=20, el optimizador fue capaz de encontrar consistentemente soluciones de alta calidad, demostrando que el CVaR proporciona una señal de costo más estable y resistente al ruido inherente al muestreo cuántico.

2.  **Superación de Desafíos del Ecosistema Qiskit:** El principal desafío del proyecto no residió en la teoría de QAOA, sino en la ingeniería de software necesaria para navegar el dinámico y fragmentado ecosistema de Qiskit. Se superaron con éxito los conflictos de versiones en Google Colab, la rápida evolución de la API de Qiskit y errores funcionales clave como la necesidad de añadir mediciones explícitas para el correcto funcionamiento de los simuladores.

**Limitaciones y Aprendizajes Clave:**

Si bien el pipeline se diseñó y codificó para ser escalable a N=100, la ejecución completa del bucle de optimización en un simulador local demostró ser computacionalmente prohibitiva. El intento de simular 100 qubits, a pesar de usar métodos eficientes como `"matrix_product_state"`, evidenció el cuello de botella de la CPU, haciendo el proceso impracticable en entornos de recursos limitados como Google Colab.

Este resultado práctico, lejos de ser un fracaso, es una conclusión valiosa: valida la necesidad fundamental de migrar a plataformas de computación en la nube para abordar problemas de esta escala. Aunque el siguiente paso lógico habría sido utilizar **Qiskit Runtime Service**, decidimos conscientemente no avanzar en esa dirección debido a las barreras prácticas que implicaba. La creación de una cuenta y la gestión de los recursos en la nube de IBM conllevan potenciales costos y tiempos de espera que escapaban al alcance y los recursos de este proyecto. Por lo tanto, optamos por enfocar nuestros esfuerzos en demostrar que nuestro pipeline era funcional y escalable en su lógica, dejando la ejecución en hardware real como un claro paso futuro.





---



## Referencias

[1] Qiskit Tutorial: *Advanced techniques for QAOA*. (https://quantum.cloud.ibm.com/docs/en/tutorials/advanced-techniques-for-qaoa)

[2] Qiskit Tutorial: *Quantum approximate optimization algorithm*. (https://quantum.cloud.ibm.com/docs/en/tutorials/quantum-approximate-optimization-algorithm)


[3] Tate, R. et al. (2023). *Warm-Started QAOA with Custom Mixers...* (q-2023-09-26-1121 (1).pdf)

[4] Lowe, A. et al. (2023). *Fast quantum circuit cutting...* (q-2023-03-02-934.pdf)